In [1]:
#!rm -rf ~/.cache/huggingface

In [2]:
!chmod -R u+rw ~/.cache/huggingface

In [3]:
!nvidia-smi

Thu Jun 26 12:37:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:06:10.0 Off |                    0 |
| N/A   31C    P0             69W /  700W |       4MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
from IPython.display import clear_output
#install vllm 
!pip install --upgrade uv
clear_output()

In [5]:
!uv pip install vllm --torch-backend=auto
!pip install -U duckduckgo-search
clear_output()

In [6]:
!pip install langchain langchain_community langchain_huggingface sentence-transformers faiss-gpu
clear_output()

In [7]:
import pandas as pd 
import os 
from tqdm import tqdm 
import os

from vllm import LLM, SamplingParams
import re 
from datetime import datetime

INFO 06-26 12:38:01 [__init__.py:244] Automatically detected platform cuda.


In [8]:
# os.listdir('../../w4-data')
test_df = pd.read_csv('../w4-data/test.csv')
submission_df = pd.read_csv('../w4-data/submission.csv')

In [9]:
#setting the llm model 
# llm = LLM(model="Qwen/Qwen3-14B")
llm = LLM(model="Qwen/Qwen3-8B")

INFO 06-26 12:38:09 [config.py:823] This model supports multiple tasks: {'generate', 'reward', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 06-26 12:38:10 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 06-26 12:38:12 [core.py:455] Waiting for init message from front-end.
INFO 06-26 12:38:12 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=F

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 06-26 12:38:18 [default_loader.py:272] Loading weights took 2.50 seconds
INFO 06-26 12:38:18 [gpu_model_runner.py:1624] Model loading took 15.2683 GiB and 4.842851 seconds
INFO 06-26 12:38:25 [backends.py:462] Using cache directory: /home/siamai/.cache/vllm/torch_compile_cache/9f391b68ca/rank_0_0 for vLLM's torch.compile
INFO 06-26 12:38:25 [backends.py:472] Dynamo bytecode transform time: 6.96 s
INFO 06-26 12:38:30 [backends.py:135] Directly load the compiled graph(s) for shape None from the cache, took 4.925 s
INFO 06-26 12:38:31 [monitor.py:34] torch.compile takes 6.96 s in total
INFO 06-26 12:38:32 [gpu_worker.py:227] Available KV cache memory: 50.28 GiB
INFO 06-26 12:38:32 [kv_cache_utils.py:715] GPU KV cache size: 366,128 tokens
INFO 06-26 12:38:32 [kv_cache_utils.py:719] Maximum concurrency for 40,960 tokens per request: 8.94x
INFO 06-26 12:38:50 [gpu_model_runner.py:2048] Graph capturing finished in 18 secs, took 0.74 GiB
INFO 06-26 12:38:50 [core.py:171] init engine (prof

In [10]:
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.8,
    min_p=0,
    top_k=20,
    max_tokens=1024
)

In [11]:
TASK_INSTRUCTIONS = """你是一位专业的语言模型代理（LLM Agent），擅长处理以下领域的问题：
伦理金融、财务管理、会计、经济学、股票变动、财务报告、固定收益、市场营销、港口管理、跨学科金融议题，以及中英文金融法规考试。

你的核心任务是：以**结构化分析 + 严密推理**为基础，**精准、简洁、高效**地完成每道题，严格遵循以下规范。

---

【题型说明】
每道题只属于**一种类型**（只会有一个答案）：
- 单项选择题（答案为 A、B、C、D 或 E）
- 金融资产判断题（答案为 "Rise" 或 "Fall"）

---

【答题步骤】
请依照以下结构进行作答：

<thinking>
1. 明确题干内容与关键词
2. 依据题目提供的明确信息，分析可能的选项逻辑（不得添加外部知识）
3. 排除错误选项，推导最可能的结论
</thinking>

<output>
"answer": [最终结论，严格为 A/B/C/D/E 或 Rise/Fall，不得附加解释]
</output>

---

【特别题型：陈述匹配题】  
当题目要求判断多个陈述（如 i)、ii)、iii)、iv)）中哪些是正确的，并选择包含这些正确陈述的选项（A/B/C/D 等）时，请遵循以下结构：  

<thinking>
1. 明确每条陈述的内容  
2. 仅依据题干信息判断每条陈述是真是伪，禁止引用外部知识  
3. 标记每条陈述为“真”或“伪”，并匹配到对应的选项  
4. 选择与所有“真”陈述一致的选项，**无需冗长分析**
</thinking>

<output>  
"answer": "A" / "B" / "C" / "D" / "E" （唯一答案）  
</output>

---

【示例图：推理结构】

***财务报告***
```mermaid
graph TD
A[阐明目的与背景] --> B[收集输入数据]
B --> C[处理数据]
C --> D[分析与解释处理后数据]
D --> E[制定并传达结论]
E --> F[进行后续跟进]
A --> |定义目标、工具和受众| B
B --> |收集经济和行业数据| C
C --> |使用比率和图表等工具| D
D --> |解释数据以得出结论| E
F --> |定期回顾和迭代| A
```
***固定收益***
```mermaid
graph TD
A[确定目的和范围] --> B[分析宏观经济条件]
B --> C[评估债券特征]
C --> D[进行风险与收益分析]
D --> E[提出投资建议]
E --> F[评估投资表现]
A --> |设定目标| B
B --> |审查利率与通胀| C
C --> |关注久期、利差| D
D --> |评估各种情景| E
```
***股权投资***
```mermaid
graph TD
A[设定投资目标] --> B[市场与行业洞察]
B --> C[行业竞争分析]
C --> D[公司评估]
D --> E[估值与风险分析]
E --> F[做出投资决策]
B --> |观察增长趋势| C
C --> |评估竞争对手地位| D
D --> |检查财务健康状况| E
E --> |将洞察整合为策略| F
```
***衍生品***
```mermaid
graph TD
A[定义目标与背景] --> B[识别衍生工具]
B --> C[理解合约规范]
C --> D[收集市场数据]
D --> E[应用估值模型]
E --> F[评估风险：市场、对手方等]
F --> G[构建收益图或策略]
G --> H[解释结果并提出建议]
H --> I[审查、监控与调整策略]
A --> |套期保值、投机、套利| B
C --> |名义金额、到期时间等特征| D
D --> |市场价格、波动率、无风险利率| E
F --> |对希腊值（Delta, Gamma等）敏感性| G
H --> |根据市场变化进行调整| I
```
***经济学***
```mermaid
graph TD
A[第1步：分解问题] -->|提取关键术语| A1[识别主题]
A1 -->|微观：供需、市场结构| A2
A1 -->|宏观：GDP、增长、政策、贸易| A3
A1 -->|货币与监管| A4
A2 --> B1[确定模型：弹性、成本曲线、停产点]
A3 --> B2[匹配AD-AS、商业周期、增长理论]
A4 --> B3[评估汇率、贸易、资本流动、监管]
B1 -->|是否涉及公式或概念？| C[数值或概念分析]
B2 --> C
B3 --> C
C -->|数值题| D1[提取数据、应用公式、验证假设]
C -->|概念题| D2[分析因果、政策影响]
D1 --> E[第4步：制定解答]
D2 --> E
E -->|构建结构化回答| E1[核心观点 + 经济逻辑]
E -->|考虑替代场景| E2[评估不同可能性]
E1 --> F[第5步：验证答案]
E2 --> F
F -->|检查逻辑、原理与假设| F1[验证一致性]
F1 -->|确保完整与清晰| F2[确认答案结构]
```
***定量方法***
```mermaid
graph TD
A[阐明目的与背景] --> B[收集输入数据]
B --> C[处理与清洗数据]
C --> D[选择定量模型与工具]
D --> E[估计参数并检验假设]
E --> F[解释结果并传达发现]
F --> G[监控并重新评估模型]
```
***投资组合管理***
```mermaid
graph TD
A[定义投资目标] --> B[建立投资约束]
B --> C[制定战略资产配置]
C --> D[加入战术性调整]
D --> E[选择并优化证券]
E --> F[执行交易与实施]
F --> G[衡量绩效与归因]
G --> H[监控风险与合规]
H --> I[再平衡并调整投资组合]
```
***另类投资***
```mermaid
graph TD
A[定义投资目标与任务] --> B[识别另类资产类别]
B --> C[进行经理和策略尽职调查]
C --> D[进行估值与定价分析]
D --> E[评估风险与流动性]
E --> F[在组合中配置另类资产]
F --> G[监控表现并再平衡]
```
***企业发行人分析***
```mermaid
graph TD
A[公司发行人概览] --> B[行业分类]
B --> C[行业趋势与竞争格局]
A --> D[财务报表分析]
D --> E[盈利能力、流动性、杠杆率]
A --> F[信用风险评估]
F --> G[评级机构与违约概率]
A --> H[资本结构与发行历史]
H --> I[债券发行与债务到期情况]
A --> J[公司治理与管理层]
J --> K[董事会质量与管理能力]
A --> L[估值与投资分析]
L --> M[DCF、相对估值、市盈率等]
```
***道德金融***
```mermaid
graph TD
A[明确道德目标] --> B[了解利益相关者影响]
B --> C[识别道德标准或准则]
C --> D[筛选金融产品或行为]
D --> E[评估利益冲突]
E --> F[应用伦理决策框架]
F --> G[记录并解释所采取的行动]
G --> H[定期审查并调整做法]
A --> |如ESG、伊斯兰金融、联合国责任投资原则| B
D --> |负面或正面筛选| E
F --> |功利主义、义务论、美德伦理学| G
```
***财务管理***
```mermaid
graph TD
A[设定财务目标] --> B[预算与预测]
B --> C[资本配置决策]
C --> D[流动性与营运资本管理]
D --> E[监控关键绩效指标与财务状况]
E --> F[根据表现与条件调整计划]
F --> G[向利益相关者汇报并优化]
B --> |运营预算、现金流预测| C
C --> |资本支出、投资回报、资本成本| D
```
***股票走势***
```mermaid
graph TD
A[识别触发事件或信号] --> B[检查技术指标]
A --> C[分析基本面新闻或财报]
B --> D[评估成交量与动量]
C --> D
D --> E[评估市场情绪]
E --> F[判断可能的价格方向]
F --> G[与历史模式交叉验证]
G --> H[做出交易或投资决策]
B --> |移动平均、RSI、MACD、布林带| D
C --> |新闻、指引、宏观、行业信息| D
```
***会计***
```mermaid
graph TD
A[识别交易或事件] --> B[确定适用的会计准则]
B --> C[分类为资产、负债、权益、收入或费用]
C --> D[应用计量规则]
D --> E[编制会计分录]
E --> F[过账至分类账与试算表]
F --> G[编制财务报表]
G --> H[分析准确性与合规性]
B --> |IFRS、GAAP| C
G --> |利润表、资产负债表、现金流量表| H
```
***市场营销（财务背景）***
```mermaid
graph TD
A[定义目标市场与营销目标] --> B[进行市场调研]
B --> C[细分并画像目标客户]
C --> D[定位金融产品]
D --> E[制定传播策略]
E --> F[启动并监测营销活动]
F --> G[衡量投资回报与客户参与度]
G --> H[根据反馈调整策略]
D --> |突出产品差异化特征| E
G --> |使用CRM与分析工具| H
```
***域外***
```mermaid
graph TD
A[识别未知或不熟悉的话题] --> B[检查关键字线索或域名提示]
B --> C[“评估与已知领域的相似性”]
C --> D[“应用一般推理或逻辑”]
D --> E[“避免没有证据的具体假设”]
E --> F[“以指出不确定性或安全概括的方式得出结论”]
A --> |这与金融相关还是完全无关？|B
D --> |使用类比或安全后备逻辑| F
```

---

【回答约束】
1. <thinking> 标签内是你的思考过程，应尽量只进行必要的推理，不得出现最终答案。
2. <output> 标签只包含两项内容：
    - 题目原文（可省略）
    - 答案，格式如下（必须严格一致）：
      - "answer": "A"
      - "answer": "B"
      - "answer": "C"
      - "answer": "D"
      - "answer": "E"
      - "answer": "Rise"
      - "answer": "Fall"
3. 严禁使用题目之外的背景信息（防止 LLM 幻觉）。
4. 信息不足时，也必须在 <thinking> 中推导最合理结论，并在 <output> 给出唯一答案。

---

今天日期是 {date}。

""".strip()

system_prompt = TASK_INSTRUCTIONS.format(date=datetime.now().strftime("%Y-%m-%d"))

In [12]:
def create_prompt(prompt):
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    return prompt

In [13]:
results = {}

for index, item in test_df.iterrows():
    if not item.id in results:
        query = item.query
        if not query.strip().endswith('คำตอบ:') and not query.strip().endswith('Answer:'):
            query += " Answer:"
        query = query.replace('คำตอบ:',"Answer:")
        prompt = create_prompt(query)

        response = llm.chat(prompt,
                            sampling_params=sampling_params,
                            chat_template_kwargs={"enable_thinking": False})
        generated_text = response[0].outputs[0].text.strip()
        print(generated_text)
        results[item.id] =  generated_text
    else:
        print("something went wrong")
    # break

INFO 06-26 12:38:53 [chat_utils.py:420] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|         | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<thinking>
1. 明确题干内容与关键词：题目讨论的是与“工作场所中增强员工能力”相关的概念，如“job enrichment”（工作丰富化）和“job enlargement”（工作扩大化），这些属于人力资源管理中的工作设计理论。
2. 分析选项逻辑：四个选项都是以“re-”开头的动词，表示某种“重新”或“再次”的过程。需要判断哪个词最符合“增强员工能力”的语境。
3. 排除错误选项：  
   - "Re-invigorating"（重振）：强调恢复活力，但不直接对应“增强能力”。  
   - "Re-flourishing"（重新繁荣）：通常用于描述整体环境或生态系统，不适用于员工能力。  
   - "Rehumanizing"（人性化）：强调回归人性化，与增强能力关系不大。  
   - "Revitalizing"（振兴）：强调恢复或增强活力，更贴近“增强员工能力”的含义。
4. 推导最可能的结论：在工作设计中，“revitalizing”最能体现通过赋予员工更多责任和决策权来增强其能力。
</thinking>

<output>
"answer": "C"
</output>


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Processed prompts:   0%|         | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<thinking>
1. 明确题干内容与关键词：题目要求判断2017-12-13日 $bac 的收盘价是上涨（Rise）还是下跌（Fall）。给出的数据包括日期、开盘价、最高价、最低价、收盘价、调整后收盘价等，以及相关社交媒体的推文内容。

2. 依据题目提供的明确信息，分析可能的选项逻辑：
   - 数据中没有2017-12-13日的记录，因此无法直接从数据中得出结论。
   - 推文内容提供了市场情绪和可能的走势暗示，但没有明确提到2017-12-13日的收盘价。
   - 推文提到“$bac is active on the market, #trading at $29.05 (+0.94%)”（2017-12-10），表明当天收盘价为 $29.05，涨幅 0.94%。
   - 2017-12-12日的收盘价为 1.3（调整后收盘价），而2017-12-11日的收盘价为 -0.4（调整后收盘价），表明价格可能在上涨。
   - 由于没有2017-12-13日的数据，只能基于12月12日的收盘价和市场情绪推断。

3. 排除错误选项，推导最可能的结论：由于没有2017-12-13日的数据，但12月12日收盘价为1.3，且市场情绪偏向上涨，推断12月13日收盘价可能继续上涨。
</thinking>

<output>
"answer": "Rise"
</output>


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|         | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<thinking>
1. 明确题干内容与关键词：题干询问“Price times quantity measures”（价格乘以数量衡量的是什么）。
2. 依据题目提供的明确信息，分析可能的选项逻辑：价格乘以数量是经济学中的基本公式，用于计算总收入或总收益。
3. 排除错误选项，推导最可能的结论：选项 C 是“total revenue”，即总收入，符合价格乘以数量的定义。
</thinking>

<output>
"answer": "C"
</output>


In [14]:
results

{'36deab86-cfd3-48b5-9bea-a36c1b0e63a8': '<thinking>\n1. 明确题干内容与关键词：题目讨论的是与“工作场所中增强员工能力”相关的概念，如“job enrichment”（工作丰富化）和“job enlargement”（工作扩大化），这些属于人力资源管理中的工作设计理论。\n2. 分析选项逻辑：四个选项都是以“re-”开头的动词，表示某种“重新”或“再次”的过程。需要判断哪个词最符合“增强员工能力”的语境。\n3. 排除错误选项：  \n   - "Re-invigorating"（重振）：强调恢复活力，但不直接对应“增强能力”。  \n   - "Re-flourishing"（重新繁荣）：通常用于描述整体环境或生态系统，不适用于员工能力。  \n   - "Rehumanizing"（人性化）：强调回归人性化，与增强能力关系不大。  \n   - "Revitalizing"（振兴）：强调恢复或增强活力，更贴近“增强员工能力”的含义。\n4. 推导最可能的结论：在工作设计中，“revitalizing”最能体现通过赋予员工更多责任和决策权来增强其能力。\n</thinking>\n\n<output>\n"answer": "C"\n</output>',
 '2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7': '<thinking>\n1. The question asks to identify the entrepreneur among the given options.\n2. Barack Obama is a former U.S. President, not an entrepreneur.\n3. James Dyson is known for inventing the bagless vacuum cleaner, making him an entrepreneur.\n4. Damien Hirst is an artist, not an entrepreneur.\n5. Mo Farah is an athlete, not an entrepreneur.\n</thinking>\n\n<output>\n"answer": "B"\n</

In [15]:
submission_df['answer'] = submission_df['id'].map(results)
submission_df

tokenize_df = submission_df.copy()

In [16]:
def extract_answer(row):
    if pd.isna(row):
        return row
    text = str(row)

    match = re.search(r'"answer"\s*:\s*"([A-E]|Rise|Fall|ขึ้น|ลง|เพิ่มขึ้น|ลดลง|เพิ่ม|ลด)"', text)
    if not match:
        match = re.search(r'"answer"\s*:\s*([A-E]|Rise|Fall|ขึ้น|ลง|เพิ่มขึ้น|ลดลง|เพิ่ม|ลด)', text)
    
    if match:
        ans = match.group(1).strip()
        if ans in ['ขึ้น', 'เพิ่มขึ้น', 'เพิ่ม']:
            return 'Rise'
        elif ans in ['ลง', 'ลดลง', 'ลด']:
            return 'Fall'
        else:
            return ans
    return 'None'

In [17]:
submission_df['answer'] = submission_df['answer'].map(extract_answer)

null_ids = submission_df[submission_df['answer'] == 'None']['id']
tokenize_df.loc[tokenize_df['id'].isin(null_ids), 'answer'] = 'None'

In [18]:
for index, i in submission_df.iterrows():
    print(i.answer)

C
B
A
C
Fall
C
B
Rise
Rise
C
A
Fall
Fall
B
C
C
B
Rise
A
Fall
C
B
B
C
Rise
D
D
Fall
C
Rise
B
A
A
A
B
D
B
B
A
A
A
A
B
C
B
C
B
Rise
A
D
D
Rise
C
Fall
A
B
Fall
A
D
Fall
Rise
B
B
B
Rise
C
C
C
B
D
Fall
A
B
B
A
D
A
D
D
A
D
D
Rise
B
B
D
B
B
B
B
A
C
A
D
A
C
C
C
D
A
A
A
C
B
Fall
A
Rise
B
D
D
D
A
D
A
A
C
D
B
Rise
C
C
D
D
C
A
B
B
A
D
C
A
D
B
B
D
C
B
B
B
Fall
Rise
D
A
Fall
D
Rise
B
D
C
Rise
B
D
D
Rise
B
D
D
C
D
A
B
A
D
B
A
C
Rise
A
B
C
B
A
C
D
Rise
B
Fall
B
B
A
C
C
C
A
D
Rise
Rise
C
C
B
A
Fall
A
Rise
B
C
B
B
B
Rise
B
B
B
A
A
C
D
C
D
C
Rise
D
D
C
D
A
Rise
B
C
Rise
D
B
B
A
A
B
Fall
C
C
C
A
D
A
D
Fall
C
D
D
Fall
B
Fall
C
D
Fall
D
Rise
C
A
C
A
C
C
Fall
B
A
B
Rise
B
B
Rise
D
Rise
A
C
Rise
C
D
Rise
D
A
D
D
B
D
D
D
Rise
Rise
Fall
C
Rise
Rise
Fall
Rise
D
Rise
C
D
A
B
D
B
D
C
B
A
B
D
Rise
C
A
A
D
D
D
B
D
A
Rise
B
Rise
C
C
C
Fall
Fall
C
D
D
B
C
D
D
Rise
D
A
C
A
B
C
C
Rise
D
A
Fall
C
B
B
B
C
Fall
D
C
Rise
B
B
B
A
B
D
C
Rise
B
A
D
Fall
B
A
D
D
Fall
A
Fall
C
Fall
C
Rise
Rise
D
A
A
Rise
D
D
D
A
B
D
D
B
Fall
A
C


In [19]:
valid_answers = ['A', 'B', 'C', 'D', 'Fall', 'Rise']

def is_valid(answer):
    return answer in valid_answers

while True:
    remain_df = submission_df[~submission_df['answer'].isin(valid_answers)]
    print(f"Remain to fix: {len(remain_df)}")
    
    if len(remain_df) == 0:
        break

    results_re = []
    ids_re = []

    for index, item in remain_df.iterrows():
        query = test_df[test_df['id'] == item.id].iloc[0]['query']
        
        if not query.strip().endswith('คำตอบ:') and not query.strip().endswith('Answer:'):
            query += " Answer:"
        query = query.replace('คำตอบ:', "Answer:")

        prompt = create_prompt(query)
        response = llm.chat(prompt,
                            sampling_params=sampling_params,
                            chat_template_kwargs={"enable_thinking": False})
        
        generated_text = response[0].outputs[0].text.strip()
        print(f"[{item.id}] => {generated_text}")
        
        ids_re.append(item.id)
        results_re.append(generated_text)

    answers_mapped = list(map(extract_answer, results_re))
    for i, ans in zip(ids_re, answers_mapped):
        if is_valid(ans):
            submission_df.loc[submission_df['id'] == i, 'answer'] = ans

Remain to fix: 0


In [20]:
submission_df['answer'].value_counts()

answer
B       105
D       105
C        98
A        90
Rise     56
Fall     45
Name: count, dtype: int64

In [21]:
submission_df.to_csv('qwen3-8b-fame-edition-1024-disiblethink-rep-new-prompt-ver002.csv',index=False)